<a href="https://colab.research.google.com/github/KirthivasanPN-hash/PodcastListeningtimeprediction/blob/main/PodcastListeningTime_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
base =  "/content/drive/MyDrive/dataset/Podcast_predict"

In [ ]:
!pip install  polars

In [3]:
import numpy
import polars as pl

import polars.selectors as cs

train_df = pl.read_csv(f"{base}/train.csv")
test_df = pl.read_csv(f"{base}/test.csv")

In [4]:
train_df.select(cs.numeric()).describe().transpose(include_header=True)

column,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8
str,str,str,str,str,str,str,str,str,str
"""statistic""","""count""","""null_count""","""mean""","""std""","""min""","""25%""","""50%""","""75%""","""max"""
"""id""","""750000.0""","""0.0""","""374999.5""","""216506.49528362884""","""0.0""","""187500.0""","""375000.0""","""562499.0""","""749999.0"""
"""Episode_Length_minutes""","""662907.0""","""87093.0""","""64.50473835100325""","""32.96960339564962""","""0.0""","""35.73""","""63.84""","""94.07""","""325.24"""
"""Host_Popularity_percentage""","""750000.0""","""0.0""","""59.8599007539""","""22.873097528069952""","""1.3""","""39.41""","""60.05""","""79.53""","""119.46"""
"""Guest_Popularity_percentage""","""603970.0""","""146030.0""","""52.23644893379306""","""28.451241399436555""","""0.0""","""28.38""","""53.58""","""76.6""","""119.91"""
"""Number_of_Ads""","""749999.0""","""1.0""","""1.3488549051398737""","""1.15113039317707""","""0.0""","""0.0""","""1.0""","""2.0""","""103.91"""
"""Listening_Time_minutes""","""750000.0""","""0.0""","""45.43740628813333""","""27.13830582797124""","""0.0""","""23.17835""","""43.37946""","""64.81158""","""119.97"""


In [ ]:
train_df.columns

['id',
 'Podcast_Name',
 'Episode_Title',
 'Episode_Length_minutes',
 'Genre',
 'Host_Popularity_percentage',
 'Publication_Day',
 'Publication_Time',
 'Guest_Popularity_percentage',
 'Number_of_Ads',
 'Episode_Sentiment',
 'Listening_Time_minutes']

 Preparing data for modeling or imputation by combining training and test sets

In [ ]:
train_df_mark = train_df.with_columns(pl.lit(1).alias("is_train")).drop(["Listening_Time_minutes"])

In [ ]:
test_df_mark = test_df.with_columns(pl.lit(0).alias("is_train"))

In [ ]:
pl.concat([train_df_mark, test_df_mark], how = "vertical")

id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,is_train
i64,str,str,f64,str,f64,str,str,f64,f64,str,i32
0,"""Mystery Matters""","""Episode 98""",null,"""True Crime""",74.81,"""Thursday""","""Night""",null,0.0,"""Positive""",1
1,"""Joke Junction""","""Episode 26""",119.8,"""Comedy""",66.95,"""Saturday""","""Afternoon""",75.95,2.0,"""Negative""",1
2,"""Study Sessions""","""Episode 16""",73.9,"""Education""",69.97,"""Tuesday""","""Evening""",8.97,0.0,"""Negative""",1
3,"""Digital Digest""","""Episode 45""",67.17,"""Technology""",57.22,"""Monday""","""Morning""",78.7,2.0,"""Positive""",1
4,"""Mind & Body""","""Episode 86""",110.51,"""Health""",80.07,"""Monday""","""Afternoon""",58.68,3.0,"""Neutral""",1
…,…,…,…,…,…,…,…,…,…,…,…
999995,"""Mind & Body""","""Episode 100""",21.05,"""Health""",65.77,"""Saturday""","""Evening""",96.4,3.0,"""Negative""",0
999996,"""Joke Junction""","""Episode 85""",85.5,"""Comedy""",41.47,"""Saturday""","""Night""",30.52,2.0,"""Negative""",0
999997,"""Joke Junction""","""Episode 63""",12.11,"""Comedy""",25.92,"""Thursday""","""Evening""",73.69,1.0,"""Neutral""",0


In [ ]:
train_for_episode_length = pl.concat([train_df_mark, test_df_mark], how = "vertical").filter(pl.col("Episode_Length_minutes").is_not_null())

In [ ]:
test_for_episode_length = pl.concat([train_df_mark, test_df_mark], how = "vertical").filter(pl.col("Episode_Length_minutes").is_null())

In [ ]:
train_for_episode_length = train_for_episode_length.filter(pl.col("Episode_Length_minutes") <= 121)

train_for_episode_length.describe()

statistic,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,is_train
str,f64,str,str,f64,str,f64,str,str,f64,f64,str,f64
"""count""",884168.0,"""884168""","""884168""",884168.0,"""884168""",884168.0,"""884168""","""884168""",718881.0,884167.0,"""884168""",884168.0
"""null_count""",0.0,"""0""","""0""",0.0,"""0""",0.0,"""0""","""0""",165287.0,1.0,"""0""",0.0
"""mean""",500196.380936,null,null,64.515109,null,59.838345,null,null,52.221719,1.353397,null,0.749751
"""std""",288688.75977,null,null,32.964307,null,22.871944,null,null,28.510346,2.475397,null,0.433157
"""min""",1.0,"""Athlete's Arena""","""Episode 1""",0.0,"""Business""",1.3,"""Friday""","""Afternoon""",0.0,0.0,"""Negative""",0.0
"""25%""",250187.0,null,null,35.74,null,39.41,null,null,28.24,0.0,null,0.0
"""50%""",500305.0,null,null,63.87,null,60.05,null,null,53.58,1.0,null,1.0
"""75%""",750244.0,null,null,94.08,null,79.51,null,null,76.68,2.0,null,1.0
"""max""",9.99999e5,"""World Watch""","""Episode 99""",120.93,"""True Crime""",119.46,"""Wednesday""","""Night""",116.82,2063.0,"""Positive""",1.0


In [ ]:
!pip install ray==2.10.0
!pip install autogluon.tabular --no-cache-dir -q
!pip install -U ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 382.4/382.4 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 kB 260.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 232.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 205.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 267.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 216.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 225.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 50.8 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widget

In [ ]:
!pip install autogluon --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.5/454.5 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.8/275.8 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from autogluon.tabular import TabularPredictor
import polars as pl

In [ ]:
train_data = train_for_episode_length.to_pandas()

In [ ]:
CALC_EPISODE = True
CALC_EPISODE = True

episode_predictor = TabularPredictor(
    path = '/content/drive/MyDrive/dataset/Podcast_predict/Autogluon_Episode_Predictor',
    label="Episode_Length_minutes",
    problem_type = 'regression',
    eval_metric = 'root_mean_squared_error',
    verbosity = 2,
    learner_kwargs = {
        'ignored_columns': [
            'id',
            ]
    })

if CALC_EPISODE:
  episode_predictor.fit(
      train_data=train_data,
      presets = 'high_quality',
      time_limit = 3600)


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.0
Python Version:     3.11.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          2
Memory Avail:       7.29 GB / 12.67 GB (57.5%)
Disk Space Avail:   12.36 GB / 15.00 GB (82.4%)
Presets specified: ['high_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Note: `save_bag_folds=False`! This will greatly reduce peak disk usage during fit (by ~8x), but runs the risk of an out-of-memory error during model refit if memory is small relative to the data size.
	You can avoid this risk by setting `save_bag_folds=True`.
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input dat

(_ray_fit pid=17903) [1000]	valid_set's rmse: 32.4963
(_ray_fit pid=17904) [1000]	valid_set's rmse: 32.3874


(_ray_fit pid=17904) 	Ran out of time, early stopping on iteration 1013. Best iteration is:
(_ray_fit pid=17904) 	[1013]	valid_set's rmse: 32.3852


(_ray_fit pid=18547) [1000]	valid_set's rmse: 32.3606
(_ray_fit pid=18580) [1000]	valid_set's rmse: 32.4532


(_ray_fit pid=18547) 	Ran out of time, early stopping on iteration 1239. Best iteration is: [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(_ray_fit pid=18547) 	[1239]	valid_set's rmse: 32.3476 [repeated 2x across cluster]


(_ray_fit pid=19181) [1000]	valid_set's rmse: 32.4784


(_ray_fit pid=19181) 	Ran out of time, early stopping on iteration 1162. Best iteration is: [repeated 2x across cluster]
(_ray_fit pid=19181) 	[1162]	valid_set's rmse: 32.4687 [repeated 2x across cluster]


(_ray_fit pid=19823) [1000]	valid_set's rmse: 32.5034 [repeated 2x across cluster]


(_ray_fit pid=19823) 	Ran out of time, early stopping on iteration 1032. Best iteration is: [repeated 2x across cluster]
(_ray_fit pid=19823) 	[1030]	valid_set's rmse: 32.5002 [repeated 2x across cluster]
(_ray_fit pid=19870) 	Ran out of time, early stopping on iteration 1096. Best iteration is:
(_ray_fit pid=19870) 	[1095]	valid_set's rmse: 32.4785
(_dystack pid=17429) 	-32.4396	 = Validation score   (-root_mean_squared_error)
(_dystack pid=17429) 	551.85s	 = Training   runtime
(_dystack pid=17429) 	201.43s	 = Validation runtime
(_dystack pid=17429) Fitting model: WeightedEnsemble_L2 ... Training model for up to 360.00s of the 253.26s of remaining time.
(_dystack pid=17429) 	Ensemble Weights: {'LightGBMXT_BAG_L1': 0.765, 'KNeighborsDist_BAG_L1': 0.235}
(_dystack pid=17429) 	-32.1197	 = Validation score   (-root_mean_squared_error)
(_dystack pid=17429) 	1.53s	 = Training   runtime
(_dystack pid=17429) 	0.02s	 = Validation runtime
(_dystack pid=17429) Fitting 106 L2 models, fit_strategy